In [ ]:
%pip install ultralytics opencv-python gdown

In [ ]:
from ultralytics import YOLO
from ultralytics.solutions import object_counter
import cv2
import os
import gdown

os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

In [ ]:
gdown.download("https://drive.google.com/uc?id=1jy05F12AE0l_b4fOAStynZFg_PaLGEPO", output="sihung.mp4")
model = YOLO("yolov8x.pt")
cap = cv2.VideoCapture("sihung.mp4")

In [ ]:
w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))
video_writer = cv2.VideoWriter(
    "result.mp4", 
    cv2.VideoWriter_fourcc(*"mp4v"),
    fps, (w, h))

counter = object_counter.ObjectCounter()  # Init Object Counter

In [ ]:
# 좌표는 좌측상단이 원점
region_points = [(0, 460), (200, 460), (250, 390), (0, 390)]
counter.set_args(view_img=True, reg_pts=region_points, classes_names=model.names, draw_tracks=True,
                 view_out_counts=False)

In [ ]:
while cap.isOpened():
    success, frame = cap.read()
    if not success:
        exit(0)
    if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    tracks = model.track(frame, persist=True, show=False)
    frame = counter.start_counting(frame, tracks)
    video_writer.write(frame)
cap.release()
cv2.destroyAllWindows()
print(f"30초간 지나간 차의 댓수 : {counter.in_counts}")

In [ ]:
import pickle
with open("sihung_result.pkl", "wb") as f:
    pickle.dump(counter, f)